# CPB - Binance US 版本 (美國用戶专用)

## 推薦：詳次筆記本已遷移到 Binance US API

✓ **無地區限制** - 完全支持美國機場  
✓ **無需 API 密鑰** - 使用公開 API  
✓ **自動重試** - API 失敗自動重新連接  
✓ **完全免費** - 20+ 幣種訓練  
✓ **GPU 加速** - Colab T4/L4 GPU 支援  
✓ **自動 Python 3.10+** - 使用 CCXT 庫  


## Step 0: 安裝必要程式

In [ ]:
# 安裝 ccxt (Binance US 交易所支援)
!pip install -q ccxt
!pip install -q ta  # 技術指標
!pip install -q scikit-learn  # 機器學習
!pip install -q torch  # PyTorch
!pip install -q python-dotenv  # 環境變數

print('\n=== 依賴安裝完成 ===')

import ccxt
import torch

print(f'CCXT 版本: {ccxt.__version__}')
print(f'PyTorch 版本: {torch.__version__}')
print(f'CUDA 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU 型號: {torch.cuda.get_device_name(0)}')

print('\n✓ 所有依賴安裝上壳')

## Step 1: 掛載 Google Drive

In [ ]:
from google.colab import drive
import os

# 掛載 Google Drive
drive.mount('/content/drive')

# 建立工作目錄
work_dir = '/content/drive/MyDrive/cpb_training'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f'✓ 工作目錄: {os.getcwd()}')
print(f'✓ GPU 檢測: {os.system("nvidia-smi -q -d Memory | grep -A 4 GPU")}')

## Step 2: Clone GitHub Repo

In [ ]:
import subprocess

# Clone repo
if not os.path.exists('cpb'):
    subprocess.run(['git', 'clone', 'https://github.com/caizongxun/cpb.git'], cwd=work_dir)
else:
    # 更新已存在的repo
    subprocess.run(['git', 'pull'], cwd=os.path.join(work_dir, 'cpb'))

os.chdir(os.path.join(work_dir, 'cpb'))
print(f'✓ 當前目錄: {os.getcwd()}')
print(f'✓ 檔案列表: {os.listdir()[:5]}')  # 顯示前 5 個檔案

## Step 3: 安裝依賴

In [ ]:
# 安裝 requirements
!pip install -q -r requirements.txt

print('✓ 依賴安裝完成！')

# 驗證 PyTorch CUDA
import torch
print(f'✓ PyTorch 版本: {torch.__version__}')
print(f'✓ CUDA 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'✓ GPU 型號: {torch.cuda.get_device_name(0)}')
    print(f'✓ GPU 記戆體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

## Step 4: 檢查 Binance US API 是否正常統作

In [ ]:
import ccxt

# 测試 Binance US API
exchange = ccxt.binanceus()

print('✓ 正在測試不同幣種...')

test_symbols = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT']

for symbol in test_symbols:
    try:
        data = exchange.fetch_ohlcv(symbol, '1h', limit=10)
        print(f'✓ {symbol}: 成功 ({len(data)} K棒)')
    except Exception as e:
        print(f'✗ {symbol}: {str(e)[:50]}...')

print('\n✓ Binance US API 正常統作！')

## Step 5: 執行完整訓練 Pipeline (Binance US 版本)

此步驟會自動執行：  
1. **Phase 1**: Binance US API 資料採集 (20+ 幣種，1h & 15m)  
2. **Phase 2**: 特徵工程 (35+ 技術指標)  
3. **Phase 3**: LSTM 模型訓練 (GPU 加速)  
4. **Phase 4**: 模型評估與報告  

**預計耗時**: 2-4 小時

In [ ]:
import sys
sys.path.insert(0, './src')

from train_us import CryptoMLPipelineUS

# 建立 Pipeline 實例 (Binance US 特化版本)
pipeline = CryptoMLPipelineUS(is_colab=True)

print('\n=== 開始執行完整 ML Pipeline ===\n')

# 執行完整 Pipeline
results_df = pipeline.run_full_pipeline(
    days_back=30,              # 採集 30 天資料
    epochs=150,                # 訓練 150 個 epoch
    skip_collection=False      # 改為 True 可跳過採集，使用本地資料
)

print('\n✓ 訓練完成！')

## Step 6: 檢視訓練結果

In [ ]:
import pandas as pd
import json

# 顯示結果 DataFrame
print('=== 訓練結果摘要 ===\n')
print(results_df)

# 讀取詳細報告
with open('./models/training_report.json', 'r') as f:
    report = json.load(f)

print('\n=== 性能統計 ===')
summary = report['summary']
print(f"訓練幣種數: {summary['total_coins']}")
print(f"平均 RMSE: {summary['avg_rmse']:.6f}")
print(f"平均 MAPE: {summary['avg_mape']:.2f}%")
print(f"平均方向準確率: {summary['avg_direction_accuracy']:.2f}%")

## Step 7: 下載模型與報告

In [ ]:
import os
from pathlib import Path

# 檢查輸出檔案
model_dir = Path('./models')
data_dir = Path('./data')

print('=== 已生成的模型 ===')
for model_file in sorted(model_dir.glob('*.pt')):
    size_mb = model_file.stat().st_size / (1024*1024)
    print(f'{model_file.name}: {size_mb:.2f} MB')

print('\n=== 報告檔案 ===')
for report_file in model_dir.glob('*.json'):
    print(f'{report_file.name}')

print('\n=== 資料採集統計 ===')
raw_data_dir = data_dir / 'raw_data'
if raw_data_dir.exists():
    csv_files = list(raw_data_dir.glob('*.csv'))
    print(f'CSV 檔案數: {len(csv_files)}')
    total_rows = sum(len(pd.read_csv(f)) for f in csv_files)
    print(f'總 K棒數: {total_rows:,}')

## 常見問題

### Q1: 為什麼用 Binance US？
**A**: Binance US 是提供美國使用者的的官方交易所，無地區限制，完全支持美國機場。

### Q2: 需要填 API 密鑰嗎？
**A**: 不需要！這个版本使用公開 API，無需認證。

### Q3: 求推的GPU推薦囉伺（適于這个項目）？
**A**: 提削:Google Colab 的 **T4 GPU** 。適幹 20+ 幣種，訓練時間約 2-3 小時。

### Q4: L4 GPU是不是更快？
**A**: 是的！L4 GPU 比 T4 快 2-3 倍。

### Q5: 修改 days_back 怎辨？
**A**: 改 `days_back=7` 可佋兩量採集 7 天資料，會快很多。預設 30 天是一个適中之選。